In [ ]:
import sys
import time

sys.path.append("../")
from calculus.pseudonym import *
from calculus.strategy import *

# TEST PROCEDURE

In [ ]:
rs = LeftmostOutermostStrategy()


def test_procedure(term_gen_func, reduction_strategy=rs):
    term = term_gen_func()
    n_term, n_steps = term.normalize(reduction_strategy)

    print(f"orig term: {term}\n")
    print(f"reduction steps: {n_steps}")
    print(f"norm term:       {n_term}")

# USEFUL TERMS

In [ ]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n = Var()
    n_ = Atom(n)
    return Lambda(
        n,
        App(
            first_term(),
            multi_app_term(
                n_, sinc_term(),
                multi_app_term(pair_term(), num_term(0), num_term(0))
            ),
        ),
    )


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# 1.
(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))
This term reduce in one step by LO, RO strategy

In [ ]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
             Lambda(x, Lambda(y, y_)),
             App(
                 Lambda(x, App(x_, x_)),
                 Lambda(x, App(x_, x_))
             )
         )


test_procedure(term_test_1())

# 2.
(ITE TRUE 3 1)

In [ ]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

# 3.
(ITE FALSE 0 4)

In [ ]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

# 4.
(PRED 0)

In [ ]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

# 5.
(PRED 5)

In [ ]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

# 6.
(PLUS 0 0)

In [ ]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

# 7.
(PLUS 2 3)

In [ ]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

# 8.
(SUBTRACT 4 1)

In [ ]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

# 9.
(SUBTRACT 3 5)

In [ ]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

# 10.
(LEQ 3 2)

In [ ]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

# 11.
(LEQ 2 5)

In [ ]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

# 12.
(EQ 2 2)

In [ ]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12())

# 13.
(EQ 1 4)

In [ ]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13())